In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error as MSE
import lightgbm as lgb
from lightgbm import LGBMRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sodapy import Socrata


In [63]:
client = Socrata("data.cityofnewyork.us", None)
results = client.get("m6nq-qud6", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [2]:
df = pd.read_csv('https://data.cityofnewyork.us/resource/djnb-wcxt.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   vendorid               998 non-null    float64
 1   lpep_pickup_datetime   1000 non-null   object 
 2   lpep_dropoff_datetime  1000 non-null   object 
 3   store_and_fwd_flag     998 non-null    object 
 4   ratecodeid             998 non-null    float64
 5   pulocationid           1000 non-null   int64  
 6   dolocationid           1000 non-null   int64  
 7   passenger_count        998 non-null    float64
 8   trip_distance          1000 non-null   float64
 9   fare_amount            1000 non-null   float64
 10  extra                  1000 non-null   float64
 11  mta_tax                1000 non-null   float64
 12  tip_amount             1000 non-null   float64
 13  tolls_amount           1000 non-null   float64
 14  ehail_fee              0 non-null      float64
 15  impro

In [31]:
ingest.green_taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   vendorid               63899 non-null   object
 1   lpep_pickup_datetime   100000 non-null  object
 2   lpep_dropoff_datetime  100000 non-null  object
 3   store_and_fwd_flag     63899 non-null   object
 4   ratecodeid             63899 non-null   object
 5   pulocationid           100000 non-null  object
 6   dolocationid           100000 non-null  object
 7   passenger_count        63899 non-null   object
 8   trip_distance          100000 non-null  object
 9   fare_amount            100000 non-null  object
 10  extra                  100000 non-null  object
 11  mta_tax                100000 non-null  object
 12  tip_amount             100000 non-null  object
 13  tolls_amount           100000 non-null  object
 14  improvement_surcharge  100000 non-null  object
 15  t

In [49]:
ingest.green_taxi_data.passenger_count.median

1.0

In [24]:
ingest.green_taxi_data.isna().sum()

vendorid                 36101
lpep_pickup_datetime         0
lpep_dropoff_datetime        0
store_and_fwd_flag       36101
ratecodeid               36101
pulocationid                 0
dolocationid                 0
passenger_count          36101
trip_distance                0
fare_amount                  0
extra                        0
mta_tax                      0
tip_amount                   0
tolls_amount                 0
improvement_surcharge        0
total_amount                 0
payment_type             36101
trip_type                36101
congestion_surcharge     36101
dtype: int64

In [4]:
# client = Socrata("data.cityofnewyork.us", None)
# results = client.get("djnb-wcxt", limit=100000)
# green_taxi_2021_api = pd.DataFrame.from_records(results)

In [ ]:
"djnb-wcxt"

In [59]:
class IngestData:

    def __init__(self, files_green=None, files_yellow=None, green_api="djnb-wcxt"):
        self.files_green = files_green
        self.files_yellow = files_yellow
        self.green_taxi_data = None
        self.yellow_taxi_data = None
        self.green_api = green_api

    def read_green_taxi_from_api(self, limit):
        client = Socrata("data.cityofnewyork.us", None)
        results = client.get(self.green_api, limit=limit)
        self.green_taxi_data= pd.DataFrame.from_records(results)   


    def read_data_green(self, files):
        self.green_taxi_data = pd.concat([pd.read_parquet(file) for file in files])
        

    def read_data_yellow(self, files):
        self.yellow_taxi_data = pd.concat([pd.read_parquet(file) for file in files])

    def fill_na(self):
        self.green_taxi_data.passenger_count.fillna(self.green_taxi_data.passenger_count.median(), inplace=True)

    def change_types(self):
        self.green_taxi_data['PULocationID'] = self.green_taxi_data['pulocationid'].astype('int64')
        self.green_taxi_data['DOLocationID'] = self.green_taxi_data['dolocationid'].astype('int64')
        self.green_taxi_data['fare_amount'] = self.green_taxi_data['fare_amount'].astype('float64')
        self.green_taxi_data['total_amount'] = self.green_taxi_data['total_amount'].astype('float64')
        self.green_taxi_data['passenger_count'] = self.green_taxi_data['passenger_count'].astype('int64')
        self.green_taxi_data['trip_distance'] = self.green_taxi_data['trip_distance'].astype('float64')
        
        
    def create_target(self):
            # self.yellow_taxi_data['tpep_pickup_datetime'] = pd.to_datetime(self.yellow_taxi_data['tpep_pickup_datetime'])
            # self.yellow_taxi_data['tpep_dropoff_datetime'] = pd.to_datetime(self.yellow_taxi_data['tpep_dropoff_datetime'])

            # self.yellow_taxi_data['trip_duration'] = self.yellow_taxi_data['tpep_dropoff_datetime'] - self.yellow_taxi_data['tpep_pickup_datetime']
            # self.yellow_taxi_data['trip_duration'] = self.yellow_taxi_data['trip_duration'].dt.total_seconds()

            self.green_taxi_data['lpep_pickup_datetime'] = pd.to_datetime(self.green_taxi_data['lpep_pickup_datetime'])
            self.green_taxi_data['lpep_dropoff_datetime'] = pd.to_datetime(self.green_taxi_data['lpep_dropoff_datetime'])

            self.green_taxi_data['trip_duration'] = self.green_taxi_data['lpep_dropoff_datetime'] - self.green_taxi_data['lpep_pickup_datetime']
            self.green_taxi_data['trip_duration'] = self.green_taxi_data['trip_duration'].dt.total_seconds()

    # def dropping_cols(self):
        # cols_to_keep = ['trip_distance', 'passenger_count', 'trip_duration', "store_and_fwd_flag", "VendorID"]

        # cols_to_drop_yellow = [col for col in self.yellow_taxi_data.columns if col not in cols_to_keep]
        # self.yellow_taxi_data.drop(columns=cols_to_drop_yellow, axis =1,  inplace=True)

        # cols_to_drop_green = [col for col in self.yellow_taxi_data.columns if col not in cols_to_keep]
        # self.green_taxi_data.drop(columns=cols_to_drop_green, axis=1,  inplace=True)

        # self.yellow_taxi_data = self.yellow_taxi_data[['trip_distance', 'passenger_count', 'trip_duration', "store_and_fwd_flag", "VendorID"]]
        # self.green_taxi_data = self.green_taxi_data[['trip_distance', 'passenger_count', 'trip_duration', "store_and_fwd_flag", "VendorID"]]

        
    def dup_and_miss(self):
        # print(f"Number of duplicated rows in yellow taxi data: {self.yellow_taxi_data.duplicated().sum()}")
        # print(f"Number of NA rows in yellow taxi data: {self.yellow_taxi_data.isna().sum().sum()}")
        print(f"Number of duplicated rows in green taxi data: {self.green_taxi_data.duplicated().sum()}")
        print(f"Number of NA rows in green taxi data: {self.green_taxi_data.isna().sum().sum()}")


    def outlier_removal(self):
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.trip_duration < 5600)]
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.trip_duration > 0)]
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.passenger_count > 0)]
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.trip_distance < 50000)]
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.fare_amount < 50000)]
        # self.yellow_taxi_data = self.yellow_taxi_data[(self.yellow_taxi_data.total_amount < 50000)]




        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.trip_duration < 5600)]
        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.trip_duration > 0)]
        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.passenger_count > 0)]
        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.trip_distance < 50000)]
        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.fare_amount < 50000)]
        self.green_taxi_data = self.green_taxi_data[(self.green_taxi_data.total_amount < 50000)]



In [60]:
ingest = IngestData()
ingest.read_green_taxi_from_api(1000)

In [61]:
ingest.fill_na()
ingest.change_types()
ingest.create_target()
ingest.dup_and_miss()
ingest.outlier_removal()


Number of duplicated rows in green taxi data: 0
Number of NA rows in green taxi data: 12
